In [90]:
import pandas as pd
import os
from re import sub
import datetime


### Configurations

In [91]:
__RAW_FOLDER_PATH__ = '../data/raw/'
__INTERIM_FOLDER_PATH__ = '../data/interim/'
__ENABLED_DATASET__ = ['Catalunya.xlsx']

In [92]:
class Dataset:

    def __init__(self, file_name: str, dataframe: pd.DataFrame) -> None:
        self.file_name = file_name
        self.dataframe = dataframe


class Utils:

    @staticmethod
    def rename_column(df: pd.DataFrame, old_column_name: str, new_column_name: str) -> pd.DataFrame:
        return df.rename({old_column_name: new_column_name}, axis=1)

    @staticmethod
    def to_snake_case(s: str) -> str:
        return '_'.join(
            sub(
                '([A-Z][a-z]+)',
                r' \1',
                sub('([A-Z]+)', r' \1', s.replace('-', ' '))
            ).split()
        ).lower()


class Cleaner:

    __FIRST_COLUMN_LABEL__ = "PRODUCTO"
    __TOTALIZER_DISCRIMINATOR__ = "SEGMENTO BIO"

    def __init__(self, data: pd.DataFrame) -> None:
        self.data = data

    def get_refined_data(self) -> pd.DataFrame:
        df = self._set_producto_column(self.data.copy())
        df = self._exclude_totes(df)
        return df

    def get_totalizer(self) -> pd.DataFrame:
        df = self._set_producto_column(self.data.copy())
        df = self._exclude_non_totes(df)
        return df

    def _set_producto_column(self, df: pd.DataFrame) -> pd.DataFrame:
        print(f'Renaming column from "{df.columns[0]}" to "{self.__FIRST_COLUMN_LABEL__}"')
        return Utils.rename_column(df, df.columns[0], self.__FIRST_COLUMN_LABEL__)

    def _exclude_totes(self, df: pd.DataFrame) -> pd.DataFrame:
        print(f'Excluding totalizer rows')
        refined_df = df[df[self.__TOTALIZER_DISCRIMINATOR__].notnull()]
        return refined_df.reset_index(drop=True)

    def _exclude_non_totes(self, df: pd.DataFrame) -> pd.DataFrame:
        print(f'Excluding non totalizer rows')
        refined_df = df[df[self.__TOTALIZER_DISCRIMINATOR__].isnull()]
        return refined_df.reset_index(drop=True)

In [93]:
dataset_file_names = os.listdir(__RAW_FOLDER_PATH__)
# dataset_file_names

In [94]:
def prepare_dataset_names(s: str) -> str:
    if s == 'C_Valenciana.xlsx':
        s = 'ComunidadValenciana.xlsx'
    return Utils.to_snake_case(s.removesuffix('.xlsx'))


def csv_to_dataset_dict(dataset_file_names: list[str]):
    datasets = {}

    for dataset_file_name in dataset_file_names:

        if __ENABLED_DATASET__ and dataset_file_name not in __ENABLED_DATASET__:
            continue

        name = prepare_dataset_names(dataset_file_name)
        df = pd.read_excel(f"{__RAW_FOLDER_PATH__}/{dataset_file_name}")

        datasets[name] = Dataset(dataset_file_name, df)

    return datasets


In [95]:
datasets = csv_to_dataset_dict(dataset_file_names)

In [96]:
datasets['catalunya'].dataframe


,Unnamed: 0,SECTOR,SECCION,CATEGORIA,FAMILIA,SEGMENTO BIO,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,TOTAL PGC,NaN,NaN,NaN,NaN,NaN,70327.94,68023.56,68846.55,72378.68,...,177706.29,183376.16,177862.27,169220.32,167775.90,171309.65,174007.15,161318.16,167443.57,168864.74
1,ALIM. Y BEBIDAS,ALIM. Y BEBIDAS,NaN,NaN,NaN,NaN,39777.17,38553.12,39145.82,41595.26,...,98944.73,100481.38,96819.19,92478.18,92111.26,93833.96,94935.21,88990.29,93129.65,94775.14
2,ALIM.SECA,ALIM. Y BEBIDAS,ALIM.SECA,NaN,NaN,NaN,19823.16,19571.58,20018.28,21250.49,...,43342.82,44952.52,44669.37,43894.22,43480.27,44579.68,46172.01,44171.38,47021.42,48898.92
3,ACEITE,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,NaN,NaN,2302.93,2191.72,2228.66,2289.00,...,3312.98,3651.12,3670.37,3336.79,3333.10,3523.74,3796.80,3552.76,3555.23,3515.62
4,GIRASOL..,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,NaN,420.76,421.54,407.75,433.43,...,487.37,548.21,559.53,523.39,503.79,537.40,578.18,500.98,523.00,537.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,RQN.AL CORTE BIO,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,RQN.AL CORTE,BIO,0.00,0.00,0.00,0.00,...,2.63,3.17,2.90,3.68,4.26,15.77,16.87,9.58,6.47,4.96
1199,RQN.AL CORTE NO BIO,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,RQN.AL CORTE,NO BIO,41.36,38.28,39.70,29.62,...,116.32,122.04,125.37,124.81,123.37,150.37,151.44,134.17,129.03,121.91
1200,RQN.EMPAQUETADO,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,RQN.EMPAQUETADO,NaN,152.73,150.41,150.97,153.27,...,421.34,414.01,422.22,444.41,426.52,415.13,415.29,394.74,398.48,402.26
1201,RQN.EMPAQUETADO BIO,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,RQN.EMPAQUETADO,BIO,2.02,2.10,1.62,1.99,...,7.66,8.34,7.88,8.58,8.97,9.65,8.85,8.31,8.17,8.08


## TRANSFORM DATAFRAME

In [97]:
cleaner = Cleaner(datasets['catalunya'].dataframe.copy())

In [98]:
df = cleaner.get_refined_data()
totalizer = cleaner.get_totalizer()

Renaming column from "Unnamed: 0" to "PRODUCTO"
Excluding totalizer rows
Renaming column from "Unnamed: 0" to "PRODUCTO"
Excluding non totalizer rows


In [99]:
df

,PRODUCTO,SECTOR,SECCION,CATEGORIA,FAMILIA,SEGMENTO BIO,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,GIRASOL.. BIO,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,BIO,0.01,0.01,0.00,0.00,...,0.11,0.12,0.17,0.15,0.15,0.17,0.14,0.14,0.14,0.16
1,GIRASOL.. NO BIO,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,NO BIO,420.75,421.53,407.75,433.42,...,487.26,548.09,559.36,523.24,503.64,537.23,578.04,500.84,522.87,537.05
2,OLIVA BIO,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,OLIVA,BIO,27.81,26.32,24.77,26.27,...,40.64,35.42,35.85,32.99,45.56,43.29,36.67,35.02,33.89,32.02
3,OLIVA NO BIO,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,OLIVA,NO BIO,1720.37,1633.57,1685.96,1714.32,...,2689.10,2962.55,2972.90,2685.73,2692.74,2847.56,3082.79,2929.48,2908.30,2856.36
4,RESTO ACEITES BIO,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,RESTO ACEITES,BIO,0.13,0.13,0.10,0.14,...,7.46,7.74,7.18,6.86,6.82,7.01,6.59,6.26,6.47,6.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,Q.REQUESON NO BIO,PROD.FRESCOS,QUESOS,REQUESON,Q.REQUESON,NO BIO,43.82,43.77,42.72,43.35,...,86.33,88.96,92.77,96.12,103.90,107.14,107.92,106.99,108.83,108.18
675,RQN.AL CORTE BIO,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,RQN.AL CORTE,BIO,0.00,0.00,0.00,0.00,...,2.63,3.17,2.90,3.68,4.26,15.77,16.87,9.58,6.47,4.96
676,RQN.AL CORTE NO BIO,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,RQN.AL CORTE,NO BIO,41.36,38.28,39.70,29.62,...,116.32,122.04,125.37,124.81,123.37,150.37,151.44,134.17,129.03,121.91
677,RQN.EMPAQUETADO BIO,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,RQN.EMPAQUETADO,BIO,2.02,2.10,1.62,1.99,...,7.66,8.34,7.88,8.58,8.97,9.65,8.85,8.31,8.17,8.08


In [100]:
totalizer

,PRODUCTO,SECTOR,SECCION,CATEGORIA,FAMILIA,SEGMENTO BIO,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,TOTAL PGC,NaN,NaN,NaN,NaN,NaN,70327.94,68023.56,68846.55,72378.68,...,177706.29,183376.16,177862.27,169220.32,167775.90,171309.65,174007.15,161318.16,167443.57,168864.74
1,ALIM. Y BEBIDAS,ALIM. Y BEBIDAS,NaN,NaN,NaN,NaN,39777.17,38553.12,39145.82,41595.26,...,98944.73,100481.38,96819.19,92478.18,92111.26,93833.96,94935.21,88990.29,93129.65,94775.14
2,ALIM.SECA,ALIM. Y BEBIDAS,ALIM.SECA,NaN,NaN,NaN,19823.16,19571.58,20018.28,21250.49,...,43342.82,44952.52,44669.37,43894.22,43480.27,44579.68,46172.01,44171.38,47021.42,48898.92
3,ACEITE,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,NaN,NaN,2302.93,2191.72,2228.66,2289.00,...,3312.98,3651.12,3670.37,3336.79,3333.10,3523.74,3796.80,3552.76,3555.23,3515.62
4,GIRASOL..,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,NaN,420.76,421.54,407.75,433.43,...,487.37,548.21,559.53,523.39,503.79,537.40,578.18,500.98,523.00,537.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,REQUESON,PROD.FRESCOS,QUESOS,REQUESON,NaN,NaN,43.82,43.77,42.72,43.35,...,86.33,88.96,92.77,96.12,103.90,107.14,107.92,106.99,108.83,108.18
520,Q.REQUESON,PROD.FRESCOS,QUESOS,REQUESON,Q.REQUESON,NaN,43.82,43.77,42.72,43.35,...,86.33,88.96,92.77,96.12,103.90,107.14,107.92,106.99,108.83,108.18
521,RESTO QUESOS NATURALES,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,NaN,NaN,194.10,188.69,190.66,182.88,...,540.28,539.22,550.49,572.90,554.15,581.27,583.61,538.48,533.98,529.13
522,RQN.AL CORTE,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,RQN.AL CORTE,NaN,41.36,38.28,39.70,29.62,...,118.95,125.21,128.27,128.49,127.63,166.14,168.31,143.74,135.50,126.87


### to fix

In [101]:
totalizer

,PRODUCTO,SECTOR,SECCION,CATEGORIA,FAMILIA,SEGMENTO BIO,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,TOTAL PGC,NaN,NaN,NaN,NaN,NaN,70327.94,68023.56,68846.55,72378.68,...,177706.29,183376.16,177862.27,169220.32,167775.90,171309.65,174007.15,161318.16,167443.57,168864.74
1,ALIM. Y BEBIDAS,ALIM. Y BEBIDAS,NaN,NaN,NaN,NaN,39777.17,38553.12,39145.82,41595.26,...,98944.73,100481.38,96819.19,92478.18,92111.26,93833.96,94935.21,88990.29,93129.65,94775.14
2,ALIM.SECA,ALIM. Y BEBIDAS,ALIM.SECA,NaN,NaN,NaN,19823.16,19571.58,20018.28,21250.49,...,43342.82,44952.52,44669.37,43894.22,43480.27,44579.68,46172.01,44171.38,47021.42,48898.92
3,ACEITE,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,NaN,NaN,2302.93,2191.72,2228.66,2289.00,...,3312.98,3651.12,3670.37,3336.79,3333.10,3523.74,3796.80,3552.76,3555.23,3515.62
4,GIRASOL..,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,NaN,420.76,421.54,407.75,433.43,...,487.37,548.21,559.53,523.39,503.79,537.40,578.18,500.98,523.00,537.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,REQUESON,PROD.FRESCOS,QUESOS,REQUESON,NaN,NaN,43.82,43.77,42.72,43.35,...,86.33,88.96,92.77,96.12,103.90,107.14,107.92,106.99,108.83,108.18
520,Q.REQUESON,PROD.FRESCOS,QUESOS,REQUESON,Q.REQUESON,NaN,43.82,43.77,42.72,43.35,...,86.33,88.96,92.77,96.12,103.90,107.14,107.92,106.99,108.83,108.18
521,RESTO QUESOS NATURALES,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,NaN,NaN,194.10,188.69,190.66,182.88,...,540.28,539.22,550.49,572.90,554.15,581.27,583.61,538.48,533.98,529.13
522,RQN.AL CORTE,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,RQN.AL CORTE,NaN,41.36,38.28,39.70,29.62,...,118.95,125.21,128.27,128.49,127.63,166.14,168.31,143.74,135.50,126.87


In [102]:
categories = totalizer[totalizer["CATEGORIA"].notnull() & totalizer["FAMILIA"].isnull()]
categories.reset_index(drop=True, inplace=True)
categories.drop(columns=['PRODUCTO', 'SECTOR', 'SECCION', 'FAMILIA', 'SEGMENTO BIO'], inplace=True)
categories

C:\Users\alfre\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,CATEGORIA,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,SEMANA 44-16,SEMANA 45-16,SEMANA 46-16,SEMANA 47-16,SEMANA 48-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,ACEITE,2302.93,2191.72,2228.66,2289.00,2113.57,2082.77,3299.20,4661.33,3011.81,...,3312.98,3651.12,3670.37,3336.79,3333.10,3523.74,3796.80,3552.76,3555.23,3515.62
1,ADITIVOS COCINA,485.41,482.40,502.81,541.91,477.30,484.85,470.82,502.79,532.50,...,1209.77,1238.96,1244.36,1201.41,1191.29,1200.97,1246.30,1153.85,1202.42,1259.35
2,ALIMENTOS INFANTILES,212.06,203.38,205.73,210.70,206.82,205.25,200.80,474.18,223.55,...,626.75,657.99,643.48,608.08,591.39,594.19,609.87,551.79,571.91,559.27
3,ALIMENTOS MASCOTAS,830.63,813.61,821.61,862.66,831.69,838.18,816.91,840.85,891.85,...,2225.05,2370.37,2307.62,2220.56,2191.05,2304.43,2323.28,2203.85,2273.26,2326.33
4,APERITIVOS,963.35,995.74,954.97,1025.04,1054.59,957.36,960.83,941.18,1042.06,...,3411.28,3184.00,2923.88,2837.83,2593.33,2658.47,2593.38,2505.42,2627.44,2634.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,QUESO RALLADO,512.76,521.15,516.15,532.17,530.66,510.80,515.06,513.99,541.41,...,1128.90,1208.63,1203.53,1230.48,1186.34,1181.59,1212.37,1146.88,1225.34,1235.59
91,QUESO TRADICIONAL,1354.67,1329.53,1315.54,1361.87,1325.43,1296.90,1303.09,1355.25,1467.09,...,3332.93,3474.00,3389.19,3432.57,3338.17,3529.90,3490.52,3317.33,3428.89,3441.78
92,QUESOS BLANCOS PASTEURIZADOS,123.40,127.95,124.45,129.30,130.03,131.01,126.46,128.73,142.93,...,336.68,356.49,356.97,367.29,360.64,372.39,384.77,363.16,368.45,359.12
93,REQUESON,43.82,43.77,42.72,43.35,41.88,43.34,44.06,42.08,44.73,...,86.33,88.96,92.77,96.12,103.90,107.14,107.92,106.99,108.83,108.18


In [103]:
categories.set_index("CATEGORIA", inplace=True)

In [104]:
categories

,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,SEMANA 44-16,SEMANA 45-16,SEMANA 46-16,SEMANA 47-16,SEMANA 48-16,SEMANA 49-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
CATEGORIA,,,,,,,,,,,,,,,,,,,,,
ACEITE,2302.93,2191.72,2228.66,2289.00,2113.57,2082.77,3299.20,4661.33,3011.81,2213.79,...,3312.98,3651.12,3670.37,3336.79,3333.10,3523.74,3796.80,3552.76,3555.23,3515.62
ADITIVOS COCINA,485.41,482.40,502.81,541.91,477.30,484.85,470.82,502.79,532.50,512.52,...,1209.77,1238.96,1244.36,1201.41,1191.29,1200.97,1246.30,1153.85,1202.42,1259.35
ALIMENTOS INFANTILES,212.06,203.38,205.73,210.70,206.82,205.25,200.80,474.18,223.55,207.60,...,626.75,657.99,643.48,608.08,591.39,594.19,609.87,551.79,571.91,559.27
ALIMENTOS MASCOTAS,830.63,813.61,821.61,862.66,831.69,838.18,816.91,840.85,891.85,853.03,...,2225.05,2370.37,2307.62,2220.56,2191.05,2304.43,2323.28,2203.85,2273.26,2326.33
APERITIVOS,963.35,995.74,954.97,1025.04,1054.59,957.36,960.83,941.18,1042.06,1101.77,...,3411.28,3184.00,2923.88,2837.83,2593.33,2658.47,2593.38,2505.42,2627.44,2634.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
QUESO RALLADO,512.76,521.15,516.15,532.17,530.66,510.80,515.06,513.99,541.41,549.87,...,1128.90,1208.63,1203.53,1230.48,1186.34,1181.59,1212.37,1146.88,1225.34,1235.59
QUESO TRADICIONAL,1354.67,1329.53,1315.54,1361.87,1325.43,1296.90,1303.09,1355.25,1467.09,1376.88,...,3332.93,3474.00,3389.19,3432.57,3338.17,3529.90,3490.52,3317.33,3428.89,3441.78
QUESOS BLANCOS PASTEURIZADOS,123.40,127.95,124.45,129.30,130.03,131.01,126.46,128.73,142.93,142.59,...,336.68,356.49,356.97,367.29,360.64,372.39,384.77,363.16,368.45,359.12


In [105]:
categoria_t = categories.transpose()

In [106]:

d = "2020-W40"
r = datetime.datetime.strptime(d + '-1', "%Y-W%W-%w")
print(r)

2020-10-05 00:00:00


In [107]:
def format_week(week: str) -> str:
    week_number = week.removeprefix("SEMANA ").split("-")
    return "20{}-W{}".format(week_number[1], week_number[0])


def reformat_week(week: str) -> datetime:
    return datetime.datetime.strptime(format_week(week) + '-1', "%Y-W%W-%w")


print(reformat_week("SEMANA 40-20"))

2020-10-05 00:00:00


In [108]:
categoria_t.index = categoria_t.index.map(reformat_week)

In [109]:
categoria_t.head()

CATEGORIA,ACEITE,ADITIVOS COCINA,ALIMENTOS INFANTILES,ALIMENTOS MASCOTAS,APERITIVOS,ARROZ,AZUCAR Y EDULCORANTES,BOLLERIA INDUSTRIAL,CACAO,CAFES,...,QUESO AZUL,QUESO BOLA,QUESO FRESCO,QUESO FUNDIDO,QUESO PASTA BLANDA,QUESO RALLADO,QUESO TRADICIONAL,QUESOS BLANCOS PASTEURIZADOS,REQUESON,RESTO QUESOS NATURALES
2016-10-03,2302.93,485.41,212.06,830.63,963.35,364.12,304.11,1692.08,520.56,1365.86,...,66.58,509.37,513.87,351.20,68.02,512.76,1354.67,123.40,43.82,194.10
2016-10-10,2191.72,482.40,203.38,813.61,995.74,357.80,314.27,1725.16,520.39,1318.65,...,66.67,483.99,505.10,341.31,69.20,521.15,1329.53,127.95,43.77,188.69
2016-10-17,2228.66,502.81,205.73,821.61,954.97,355.71,331.11,1880.66,530.73,1362.94,...,64.86,481.32,498.29,346.66,68.27,516.15,1315.54,124.45,42.72,190.66
2016-10-24,2289.00,541.91,210.70,862.66,1025.04,369.98,367.59,2026.92,534.90,1406.60,...,67.90,490.24,502.97,355.83,70.67,532.17,1361.87,129.30,43.35,182.88
2016-10-31,2113.57,477.30,206.82,831.69,1054.59,365.46,325.75,1793.27,501.73,1391.35,...,69.08,471.02,479.10,338.57,70.14,530.66,1325.43,130.03,41.88,191.04


In [111]:
categoria_t["COMMUNITY"] =  "CATALUNYA"

In [112]:
categoria_t.to_csv(f"{__INTERIM_FOLDER_PATH__}catalunya.csv")